In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import fsolve
from scipy.stats import norm
from tqdm.notebook import tqdm

In [3]:
sigma11 = 0.06
sigma12 = 0.00
sigma21 = 0.04
sigma22 = 0.03

kappa1  = 0.05
kappa2  = 0.06

sigmas = np.array([[sigma11, sigma12], 
                   [sigma21, sigma22]])
kappas = np.array([kappa1, kappa2])

ycrv_t     = [0.01, 1, 2, 3, 5, 10, 20, 30]
ycrv_rates = [0.03,0.031, 0.033, 0.037, 0.04, 0.042, 0.046, 0.05]

In [3]:
def g(t, sigmas, kappas):
    return sigmas*np.exp(t*kappas)

def h(t, kappas):
    return np.exp(-t*kappas)

def H(t, kappas):
    return np.diag(h(t,kappas))
    
def sigmax(sigmas):
    sig_x = np.copy(sigmas)
    sig_x[0,1] = 0 #sigma12 = 0
    return sig_x

def sigmap(t, T, sigmas, kappas):
    h_integral = ((np.exp(-kappas*t)-np.exp(-kappas*T))/kappas)
    return g(t, sigmas, kappas) @ h_integral

def kappa_mat(kappas):
    return np.diag(kappas)

def M(t, T, kappas):
    return np.exp(-(T-t)*kappas)

def G(t, T, kappas):
    return (1.0-np.exp(-(T-t)*kappas))/kappas

def y(t, sigmas, kappas):
    kappa1, kappa2 = kappas
    sigma11, sigma12, sigma21, sigma22 = sigmas.flatten()
    gg11 = (np.exp(2*kappa1*t)-1)*(sigma11**2 + sigma21**2)/(2*kappa1)
    gg12 = (np.exp(sum(kappas)*t)-1)*(sigma11*sigma12+sigma21*sigma22)/sum(kappas)
    gg21 = gg12
    gg22 = (np.exp(2*kappa2*t)-1)*(sigma12**2 + sigma22**2)/(2*kappa2)
    gg_integral = np.array([[gg11, gg12], [gg21, gg22]])
    return H(t, kappas) @ gg_integral @ H(t, kappas)

def P(t,T, Pt, PT, sigmas, kappas, x):
    ''' 
        Function computes bond price given x = x(t): P(t, T| x(t) = x)
    '''
    A = -0.5* G(t, T, kappas) @ y(t, sigmas, kappas) @ G(t, T, kappas)
    return PT/Pt*np.exp(-G(t, T, kappas) @ np.array(x) + A)

def var(T, sigmas, kappas):
    sigma_square_sums = (np.transpose(sigmas)**2).sum(axis = 1)
    return (1-np.exp(-2*kappas*T))/(2*kappas)*sigma_square_sums

def cov(T, sigmas, kappas):
    kappa_integral = (1-np.exp(-sum(kappas)*T))/sum(kappas)
    return sum(sigmas.prod(axis = 1))*kappa_integral
    
def mu1_cond(T, sigmas, kappas, x2):
    return cov(T, sigmas, kappas)/var(T,sigmas,kappas)[1]*x2

def s1_sq_cond(T, sigmas, kappas):
    var1, var2 = var(T, sigmas, kappas)
    return var1 - (cov(T,sigmas, kappas)**2)/var2
    
def v(t, sigmas, kappas):
    return np.sum(y(t, sigmas, kappas), axis = 1)

def vT0(t, T0, sigmas, kappas):
    return v(t, sigmas, kappas) - np.transpose(sigmas) @ sigmap(t, T0, sigmas, kappas)

In [4]:
def disc(t, ycrv_t, ycrv_rates, freq = 2):
    return 1.0/(1.0+np.interp(np.array(t), ycrv_t, ycrv_rates)/freq)**(freq * np.array(t))


def swap_value(w, K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates, x):
    # w=-1 payer swap; w=1 receiver swap
    yearfrac = 1/freq
    coupon_times = np.arange(T0+yearfrac, TN+0.0001, yearfrac)
    PT0 = disc(T0, ycrv_t, ycrv_rates)
    PTi = disc(coupon_times, ycrv_t, ycrv_rates)
    bond_prices = np.zeros_like(coupon_times)
    for i in range(len(coupon_times)):
        bond_prices[i] = P(T0, coupon_times[i], PT0, PTi[i], sigmas, kappas, x)
    return -w*1 + w*bond_prices[-1] + w*K*yearfrac*bond_prices.sum()

def get_strikes(w, K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates, x2):
    '''
        For a given x2, find critical x1 = x1*(x2) and compute strikes Ki=P(T0,Ti, x1, x2) 
        for Jamshidian decomposition
    '''
    x1_crit = fsolve(lambda x1: swap_value(w, K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates, [x1, x2]),
                     x0 = [0])[0]
    x = [x1_crit, x2]
    yearfrac = 1/freq
    coupon_times = np.arange(T0+yearfrac, TN+0.0001, yearfrac)
    Ki = []
    PT0 = disc(T0, ycrv_t, ycrv_rates, freq = 2)
    for Ti in coupon_times:
        Ki.append(P(T0, Ti, PT0, disc(Ti, ycrv_t, ycrv_rates, freq = 2), sigmas, kappas, x))
    return coupon_times, Ki

def bond_put_option(T, s, K, sigmas, kappas, ycrv_t, ycrv_rates, x2):
    df        = lambda t: disc(t, ycrv_t, ycrv_rates, freq = 2)
    variances = var(T, sigmas, kappas)
    covs      = cov(T, sigmas, kappas)
    mu1       = mu1_cond(T, sigmas, kappas, x2)
    s1_sq     = s1_sq_cond(T, sigmas, kappas)
    G1, G2    = G(T, s, kappas)
    omega     = -mu1*G1 + 0.5*(G1**2)*s1_sq
    A         = -0.5* G(T, s, kappas) @ y(T, sigmas, kappas) @ G(T, s, kappas)
    
    K_star    = df(T)/df(s)*np.exp(-A+x2*G2)*K
    d = lambda sign: (omega-np.log(K_star)+sign*0.5*(G1**2)*s1_sq)/(G1*np.sqrt(s1_sq))
    
    value = df(s)*np.exp(A-x2*G2)*(K_star*norm.cdf(-d(-1))-np.exp(omega)*norm.cdf(-d(1)))
    return value

def bond_call_option(T, s, K, sigmas, kappas, ycrv_t, ycrv_rates, x2):
    df        = lambda t: disc(t, ycrv_t, ycrv_rates, freq = 2)
    variances = var(T, sigmas, kappas)
    covs      = cov(T, sigmas, kappas)
    mu1       = mu1_cond(T, sigmas, kappas, x2)
    s1_sq     = s1_sq_cond(T, sigmas, kappas)
    G1, G2    = G(T, s, kappas)
    omega     = -mu1*G1 + 0.5*(G1**2)*s1_sq
    A         = -0.5* G(T, s, kappas) @ y(T, sigmas, kappas) @ G(T, s, kappas)
    
    K_star    = df(T)/df(s)*np.exp(-A+x2*G2)*K
    d = lambda sign: (omega-np.log(K_star)+sign*0.5*(G1**2)*s1_sq)/(G1*np.sqrt(s1_sq))
    
    value = df(s)*np.exp(A-x2*G2)*(np.exp(omega)*norm.cdf(d(1))-K_star*norm.cdf(d(-1)))
    return value

def payer_swaption(K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates):
    
    def payer_swaption_x2_cond(K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates, x2):
        coupon_times, strikes = get_strikes(-1, K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates, x2)
        bond_option = []
        for Ti, Ki in zip(coupon_times, strikes):
            bond_option.append(bond_put_option(T0, Ti, Ki, sigmas, kappas, ycrv_t, ycrv_rates, x2))    
        var_x2 = var(T0, sigmas, kappas)[1]
        return (bond_option[-1]+K*1/freq*sum(bond_option))/np.sqrt(var_x2)*norm.pdf(x2/np.sqrt(var_x2))
        
    x2_grid = np.linspace(-0.35, 0.35, 800)
    swaption_val_x2 = []
    for x2 in x2_grid:
        swaption_val_x2.append(payer_swaption_x2_cond(K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates, x2))
    
    return np.trapz(swaption_val_x2, x2_grid)

def receiver_swaption(K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates):
    
    def receiver_swaption_x2_cond(K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates, x2):
        coupon_times, strikes = get_strikes(-1, K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates, x2)
        bond_option = []
        for Ti, Ki in zip(coupon_times, strikes):
            bond_option.append(bond_call_option(T0, Ti, Ki, sigmas, kappas, ycrv_t, ycrv_rates, x2))    
        var_x2 = var(T0, sigmas, kappas)[1]
        return (bond_option[-1]+K*1/freq*sum(bond_option))/np.sqrt(var_x2)*norm.pdf(x2/np.sqrt(var_x2))
        
    x2_grid = np.linspace(-0.35, 0.35, 800)
    swaption_val_x2 = []
    for x2 in x2_grid:
        swaption_val_x2.append(receiver_swaption_x2_cond(K, T0, TN, freq, sigmas, kappas, ycrv_t, ycrv_rates, x2))
    
    return np.trapz(swaption_val_x2, x2_grid)

def q(t, T0, TN, sigmas, kappas, ycrv_t, ycrv_rates, x = [0.0, 0.0], freq = 2):
    Pt = disc(t, ycrv_t, ycrv_rates, freq = 2)
    yearfrac = 1/freq
    times = np.arange(T0, TN+0.0001, yearfrac)
    bond_prices = []
    G_vals = []
    for T in times:
        bond_prices.append(P(t,T, Pt, disc(T, ycrv_t, ycrv_rates, freq = 2), sigmas, kappas, x))
        G_vals.append(G(t, T, kappas))
    bond_prices = np.array(bond_prices)
    G_vals = np.array(G_vals)
    G1_vals, G2_vals = G_vals[:,0], G_vals[:,1]
    
    A = yearfrac * bond_prices[1:].sum()
    S = (bond_prices[0] - bond_prices[-1])/A
    
    q1 =(bond_prices[0]*G1_vals[0]-bond_prices[-1]*G1_vals[-1])/A-S*yearfrac*(G1_vals[1:]*bond_prices[1:]).sum()/A
    q2 =(bond_prices[0]*G2_vals[0]-bond_prices[-1]*G2_vals[-1])/A-S*yearfrac*(G2_vals[1:]*bond_prices[1:]).sum()/A
    
    return np.array([q1, q2])
    
def payer_swaption_approx(K, T0, TN, sigmas, kappas, ycrv_t, ycrv_rates, x = [0.0, 0.0], freq = 2):
    
    integrand = lambda t: np.linalg.norm(q(t,T0,TN,sigmas,kappas,ycrv_t,ycrv_rates,x,freq) @ np.transpose(sigmas))**2
    t_grid = np.linspace(0,T0, 100)
    integrand_val = []
    for t in t_grid:
        integrand_val.append(integrand(t))
    v = np.trapz(integrand_val, t_grid)
    
    yearfrac = 1/freq
    times = np.arange(T0, TN+0.0001, yearfrac)
    bond_prices = disc(times, ycrv_t, ycrv_rates, freq = 2)
    A = yearfrac * bond_prices[1:].sum()
    S = (bond_prices[0]-bond_prices[-1])/A
    d = (S - K)/np.sqrt(v)
    return A*((S-K)*norm.cdf(d)+np.sqrt(v)*norm.pdf(d))

def receiver_swaption_approx(K, T0, TN, sigmas, kappas, ycrv_t, ycrv_rates, x = [0.0, 0.0], freq = 2):
    
    integrand = lambda t: np.linalg.norm(q(t,T0,TN,sigmas,kappas,ycrv_t,ycrv_rates,x,freq) @ np.transpose(sigmas))**2
    t_grid = np.linspace(0,T0, 100)
    integrand_val = []
    for t in t_grid:
        integrand_val.append(integrand(t))
    v = np.trapz(integrand_val, t_grid)
    
    yearfrac = 1/freq
    times = np.arange(T0, TN+0.0001, yearfrac)
    bond_prices = disc(times, ycrv_t, ycrv_rates, freq = 2)
    A = yearfrac * bond_prices[1:].sum()
    S = (bond_prices[0]-bond_prices[-1])/A
    d = (S - K)/np.sqrt(v)
    return A*((K-S)*norm.cdf(-d)+np.sqrt(v)*norm.pdf(-d))

In [7]:
swap_value(-1, 0.058, 2, 7, 2, sigmas, kappas, ycrv_t, ycrv_rates, x = [0.00, 0.00])
#find_critical_x1(-1, 0.058, 2, 7, 2, sigmas, kappas, ycrv_t, ycrv_rates, x2 = 0.0)
get_strikes(-1, 0.058, 2, 7, 2, sigmas, kappas, ycrv_t, ycrv_rates, x2 = 0)
bond_put_option(T=1, s=5, K=0.79, sigmas=sigmas, kappas=kappas, ycrv_t=ycrv_t, ycrv_rates=ycrv_rates, x2=0.0)
print(payer_swaption(0.058, 2, 7, 2, sigmas, kappas, ycrv_t, ycrv_rates))
print(receiver_swaption(0.058, 2, 7, 2, sigmas, kappas, ycrv_t, ycrv_rates))
#q(0, 2, 7, sigmas, kappas, ycrv_t, ycrv_rates, x = [0.0, 0.0], freq = 2)
print(payer_swaption_approx(0.058, 2, 7, sigmas, kappas, ycrv_t, ycrv_rates, x = [0.0, 0.0], freq = 2))
print(receiver_swaption_approx(0.058, 2, 7, sigmas, kappas, ycrv_t, ycrv_rates, x = [0.0, 0.0], freq = 2))

0.15669065319192452
0.21512913800078298
0.1582501718375755
0.21668865664643377


In [46]:
dt = 1/252
T0 = 2
TN = 7
K  = 0.058
n_simuls = 10000

def simulate_x(n_simuls, T0, sigmas, kappas, dt):
    t_grid = np.arange(dt, T0+0.0001, dt)
    dW = lambda: np.sqrt(dt)*np.random.normal(size = 2)
    x_final = []
    
    for simul in tqdm(range(n_simuls)):
        x = np.array([0.0, 0.0])
        for t in t_grid:
            x += (vT0(t, T0, sigmas, kappas) - kappas * x)*dt + np.transpose(sigmas) @ dW()
        x_final.append(x)
    return np.array(x_final)

def payoff(x, T0, TN, K, ycrv_t, ycrv_rates, freq = 2):
    yearfrac = 1/freq
    times = np.arange(T0+yearfrac, TN+0.0001, yearfrac)
    PT0 = disc(T0, ycrv_t, ycrv_rates, freq = 2)
    bond_prices = []
    for Ti in times:
        bond_prices.append(P(T0,Ti, PT0, disc(Ti, ycrv_t, ycrv_rates, 2), sigmas, kappas, x))
    return np.max([-(1 - bond_prices[-1] - K*yearfrac*sum(bond_prices)),0])

payoffs = []
x_final = simulate_x(n_simuls, T0, sigmas, kappas, dt)
for i in range(len(x_final)):
    payoffs.append(payoff(x_final[i,:]), 2, 7, 0.058, ycrv_t, ycrv_rates, freq = 2)
    
pd.Series(payoffs).mean()*disc(T0, ycrv_t, ycrv_rates,2)

TypeError: payoff() missing 5 required positional arguments: 'T0', 'TN', 'K', 'ycrv_t', and 'ycrv_rates'

In [49]:
for i in range(len(x_final)):
    payoffs.append(payoff(x_final[i,:], 2, 7, 0.058, ycrv_t, ycrv_rates, freq = 2))

In [48]:
payoff(x_final[i,:])

TypeError: payoff() missing 5 required positional arguments: 'T0', 'TN', 'K', 'ycrv_t', and 'ycrv_rates'

In [50]:
pd.Series(payoffs).mean()*disc(T0, ycrv_t, ycrv_rates,2)

0.21687733351572594

In [ ]:
(1-np.exp(-2*kappas*10))/(2*kappas)

In [ ]:
y(1, sigmas, kappas)

In [ ]:
vT0(1, 3, sigmas, kappas)

In [ ]:
np.transpose(sigmas) @ sigmap(1, 3, sigmas, kappas)

In [ ]:
sigmap(1, 3, sigmas, kappas)

In [ ]:
sigmap(1, 10, sigmas, kappas)

In [ ]:
np.linalg.norm([1, 2])

In [2]:
np.array([[1, 1],[1, 1]])*np.array([1, 2])

array([[1, 2],
       [1, 2]])

In [5]:
def get_df(t, ycrv_t, ycrv_rates, ycrv_freq):
    ''' 
        discout factor function
    '''
    return 1.0/(1.0+np.interp(t, ycrv_t, ycrv_rates)/ycrv_freq)**(t*ycrv_freq)

def fwd_swap_rate(T_start, T_end, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq):
    ''' 
        function calculates forward swap rate R
        for a swap starting at T_start and ending at T_end
    '''
    
    coupon_dates = np.arange(T_start, T_end+0.0001, swap_yearfrac)
    df = get_df(coupon_dates, ycrv_t, ycrv_rates, ycrv_freq)
    
    f = (df[0] - df[-1])/(swap_yearfrac * np.sum(df[1:]))
    return f

def swaption(T_start, T_end, K, IRS_type, sigma, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq, model, shift = 0):
    '''
        calculate swaption price. The underlying is a swap starting at T_start and ending at T_end
        where T_start is also the expiry of the swaption
        model is either normal, or (shifted)lognormal; normal model takes no shift
    '''
    f            = fwd_swap_rate(T_start, T_end, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq)
    coupon_dates = np.arange(T_start, T_end+0.0001, swap_yearfrac)
    df           = get_df(coupon_dates, ycrv_t, ycrv_rates, ycrv_freq)
    annuity      = swap_yearfrac*np.sum(df[1:])
    
    w = {'payer':-1, 'receiver':1}[IRS_type.lower()]
    
    if model.lower() == 'lognormal':
        d1      = (np.log((f+shift)/(K+shift)) + (0.5* sigma**2 * T_start))/(sigma*np.sqrt(T_start))
        d2      = d1 - sigma*np.sqrt(T_start)
        price   = (w*(K+shift)*norm.cdf(-w*d2) - w*(R+shift)*norm.cdf(-w*d1))*annuity
    elif model.lower() == 'normal':
        d       = -w * (f-K)*(sigma * np.sqrt(T_start))
        price   = sigma * np.sqrt(T_start) * (d * norm.cdf(d) + norm.pdf(d)) * annuity
    else:
        print(f'Swaption pricing: unknown model {model}')
      
    return price

In [6]:
ycrv = pd.read_csv('IR_curve_EUR_2018_12_31.csv')
vols = pd.read_csv('normal_volsurface_2018_12_31.csv')
vols = vols.rename(columns = {'expiry/tenor':'expiry'}).set_index('expiry').rename_axis('tenor', axis = 1)
ycrv.head()

,maturity,value
0,0.019231,-0.004668
1,0.083333,-0.003982
2,0.166667,-0.003967
3,0.250000,-0.003374
4,0.500000,-0.003111


In [7]:
import xlwings as xw
stacked_vols = vols.stack().reset_index().rename(columns = {0:'sigma_normal'})
stacked_vols['tenor'] = stacked_vols['tenor'].astype(float) 
stacked_vols['TN'] = stacked_vols.eval('expiry+tenor')
selected_vols = stacked_vols.query('TN>=10 and TN<=15').rename(columns = {'expiry':'T0'}).drop('tenor', axis = 1)
selected_vols = selected_vols[['T0', 'TN', 'sigma_normal']]
#selected_vols

In [8]:
def get_err(params, df_vol_quotes, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq):
    kappa1, kappa2, sigma1, sigma2, rho = params
    
    sigmas = np.array([[sigma1, sigma2*rho],[0, sigma2*np.sqrt(1-rho**2)]])
    kappas = np.array([kappa1, kappa2])
    
    market_vals, model_vals = [], []
    for _, row in df_vol_quotes.iterrows():
        T0, TN, sigma_normal = row 
        f = fwd_swap_rate(T0, TN, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq)
        mkt_val = swaption(T0, TN, f, 'payer', sigma_normal, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq, 'normal')
        model_val = payer_swaption_approx(f, T0, TN, sigmas, kappas, ycrv_t, ycrv_rates, x = [0.0, 0.0], freq = 2)
        market_vals.append(mkt_val)
        model_vals.append(model_val)
    sq_err = np.sqrt(((np.array(model_vals)-np.array(market_vals))**2).mean())
    print(f'params {params}, error: {sq_err}')
    return sq_err

In [10]:
def get_err_lsq(params, df_vol_quotes, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq):
    kappa1, kappa2, sigma1, sigma2, rho = params
    
    sigmas = np.array([[sigma1, sigma2*rho],[0, sigma2*np.sqrt(1-rho**2)]])
    kappas = np.array([kappa1, kappa2])
    
    market_vals, model_vals = [], []
    for _, row in df_vol_quotes.iterrows():
        T0, TN, sigma_normal = row 
        f = fwd_swap_rate(T0, TN, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq)
        mkt_val = swaption(T0, TN, f, 'payer', sigma_normal, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq, 'normal')
        model_val = payer_swaption_approx(f, T0, TN, sigmas, kappas, ycrv_t, ycrv_rates, x = [0.0, 0.0], freq = 2)
        market_vals.append(mkt_val)
        model_vals.append(model_val)
    sq_err = np.sqrt(((np.array(model_vals)-np.array(market_vals))**2).mean())
    print(f'params {params}, error: {sq_err}')
    return (np.array(model_vals)-np.array(market_vals))

In [9]:
from scipy.optimize import minimize
fun = lambda params: get_err(params, selected_vols, 0.5, ycrv['maturity'], ycrv['value'], ycrv_freq = 99999)
res = minimize(fun, x0 = [0.015, 0.013, 0.00627, 0.00148, 0.9], tol=1e-6, options = {'disp':True},
              bounds = ((0.00001, None), (0.00001, None), (0.00001, None), (0.00001, None), (-0.9999, 0.9999)))

params [0.015   0.013   0.00627 0.00148 0.9    ], error: 0.005673959505805713
params [0.01500001 0.013      0.00627    0.00148    0.9       ], error: 0.005673958020897882
params [0.015      0.01300001 0.00627    0.00148    0.9       ], error: 0.005673959172401602
params [0.015      0.013      0.00627001 0.00148    0.9       ], error: 0.005673999463727488
params [0.015      0.013      0.00627    0.00148001 0.9       ], error: 0.005673997407776944
params [0.015      0.013      0.00627    0.00148    0.90000001], error: 0.005673959555060744
params [1.63490783e-01 4.63404112e-02 1.00000000e-05 1.00000000e-05
 8.95074497e-01], error: 0.03078145841215198
params [1.63490793e-01 4.63404112e-02 1.00000000e-05 1.00000000e-05
 8.95074497e-01], error: 0.03078145841234176
params [1.63490783e-01 4.63404212e-02 1.00000000e-05 1.00000000e-05
 8.95074497e-01], error: 0.030781458412564872
params [1.63490783e-01 4.63404112e-02 1.00100000e-05 1.00000000e-05
 8.95074497e-01], error: 0.030781454281953013
par

KeyboardInterrupt: 

In [18]:
from scipy.optimize import least_squares
fun = lambda params: get_err_lsq(params, selected_vols, 0.5, ycrv['maturity'], ycrv['value'], ycrv_freq = 99999)
res = least_squares(fun, x0 = [0.03, 0.36, 0.01, 0.015, -0.9], verbose=2,
                   bounds = ([0.0001, 0.0001, 0.0001, 0.0001, -0.9999], [np.inf, np.inf, np.inf, np.inf, 0.9999]))

params [ 0.03   0.36   0.01   0.015 -0.9  ], error: 0.0020495752681858074
params [ 0.03000001  0.36        0.01        0.015      -0.9       ], error: 0.0020495731101620738
params [ 0.03        0.36000001  0.01        0.015      -0.9       ], error: 0.002049575597634224
params [ 0.03        0.36        0.01000001  0.015      -0.9       ], error: 0.0020496113604249115
params [ 0.03        0.36        0.01        0.01500001 -0.9       ], error: 0.002049569037345815
params [ 0.03        0.36        0.01        0.015      -0.90000001], error: 0.0020495750922734086
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.1552e-04                                    4.71e-02    
params [ 0.02655129  0.38312182  0.01029986  0.02226992 -0.89373751], error: 0.0013465544901463946
params [ 0.02655131  0.38312182  0.01029986  0.02226992 -0.89373751], error: 0.001346552399311061
params [ 0.02655129  0.38312184  0.01029986  0.0222699

params [ 0.06977187  0.58041803  0.01343077  0.04004461 -0.96988637], error: 0.00035253815602386267
params [ 0.06977186  0.58041805  0.01343077  0.04004461 -0.96988637], error: 0.000352538060536943
params [ 0.06977186  0.58041803  0.01343078  0.04004461 -0.96988637], error: 0.00035253687344139004
params [ 0.06977186  0.58041803  0.01343077  0.04004463 -0.96988637], error: 0.00035253817537034195
params [ 0.06977186  0.58041803  0.01343077  0.04004461 -0.96988639], error: 0.0003525380826631284
      11             17         3.4178e-06      9.25e-08       8.16e-03       1.56e-03    
params [ 0.07299111  0.56518005  0.01389276  0.03997822 -0.97198791], error: 0.00034875458771446255
params [ 0.07299113  0.56518005  0.01389276  0.03997822 -0.97198791], error: 0.0003487550105202302
params [ 0.07299111  0.56518006  0.01389276  0.03997822 -0.97198791], error: 0.0003487545248317669
params [ 0.07299111  0.56518005  0.01389278  0.03997822 -0.97198791], error: 0.00034874948824476377
params [ 0.072

KeyboardInterrupt: 

In [32]:
def get_err_lsq_lmfit(params, df_vol_quotes, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq):
    parvals = params.valuesdict()
    kappa1 = parvals['kappa1']
    kappa2 = parvals['kappa2']
    sigma1 = parvals['sigma1']
    sigma2 = parvals['sigma2']
    rho    = parvals['rho']
    print(f'{kappa1},{kappa2},{sigma1},{sigma2},{rho}')
    sigmas = np.array([[sigma1, sigma2*rho],[0, sigma2*np.sqrt(1-rho**2)]])
    kappas = np.array([kappa1, kappa2])
    
    market_vals, model_vals = [], []
    for _, row in df_vol_quotes.iterrows():
        T0, TN, sigma_normal = row 
        f = fwd_swap_rate(T0, TN, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq)
        mkt_val = swaption(T0, TN, f, 'payer', sigma_normal, swap_yearfrac, ycrv_t, ycrv_rates, ycrv_freq, 'normal')
        model_val = payer_swaption_approx(f, T0, TN, sigmas, kappas, ycrv_t, ycrv_rates, x = [0.0, 0.0], freq = 2)
        market_vals.append(mkt_val)
        model_vals.append(model_val)
    sq_err = np.sqrt(((np.array(model_vals)-np.array(market_vals))**2).mean())
    #print(f'params {params}, error: {sq_err}')
    return (np.array(model_vals)-np.array(market_vals))

In [33]:
from lmfit import Parameters, Minimizer
params = Parameters()
params.add_many(('kappa1', 0.03,  True, 0.0001, 1.0, None, None),
                ('kappa2', 0.03,  True, 0.0001, 1.0, None, None),
                ('sigma1', 0.01,  True, 0.0001, 1.0, None, None),
                ('sigma2', 0.015, True, 0.0001, 1.0, None, None),
                ('rho',    0.0,   True,-0.9999,0.9999, None, None))
fun = lambda params: get_err_lsq_lmfit(params, selected_vols, 0.5, ycrv['maturity'], ycrv['value'], ycrv_freq = 99999)
fitter = Minimizer(fun, params = params)
fitter.minimize()

0.030000000000000027,0.030000000000000027,0.010000000000000009,0.014999999999999902,0.0
0.030000000000000027,0.030000000000000027,0.010000000000000009,0.014999999999999902,0.0
0.030000000000000027,0.030000000000000027,0.010000000000000009,0.014999999999999902,0.0
0.030000000878154465,0.030000000000000027,0.010000000000000009,0.014999999999999902,0.0
0.030000000000000027,0.030000000878154465,0.010000000000000009,0.014999999999999902,0.0
0.030000000000000027,0.030000000000000027,0.010000000293596933,0.014999999999999902,0.0
0.030000000000000027,0.030000000000000027,0.010000000000000009,0.015000000440794858,0.0
0.030000000000000027,0.030000000000000027,0.010000000000000009,0.014999999999999902,1.4899671052504004e-08
218.70374219429746,96.51562200030271,9.586841673672957,4.653105017197066,-0.6080791599378439


ValueError: NaN values detected in your input data or the output of your objective/model function - fitting algorithms cannot handle this! Please read https://lmfit.github.io/lmfit-py/faq.html#i-get-errors-from-nan-in-my-fit-what-can-i-do for more information.

NameError: name 'parvals' is not defined